In [2]:
import astropy.units as u
from hydrad_tools.configure import Configure

In [6]:
config = Configure.load_config('/Users/willbarnes/Documents/work/codes/hydrad_tools/defaults.asdf')
config['general']['total_time'] = 5e3*u.s
config['initial_conditions']['heating_location'] = 25*u.Mm
config['initial_conditions']['heating_scale_height'] = 1e300*u.cm
config['heating']['background_heating'] = False
config['heating']['events'] = [
    {
        'time_start': 0*u.s, 'rise_duration': 100*u.s, 'decay_duration': 100*u.s,'total_duration': 200*u.s,
        'location': 50*u.Mm, 'scale_height': 1e300*u.cm, 'rate': 0.1*u.erg/u.s/(u.cm**3)
    }
]
config['general']['initial_amr_file'] = 'Results/profile8.amr'

In [3]:
ic_template = """// ****
// *
// * #defines for configuring the hydrostatic model
// *
// * (c) Dr. Stephen J. Bradshaw
// *
// * Source code generated by hydrad_tools on {{ date }}
// *
// ****

// **** Output ****
// **** End of Output ****

// **** Physics ****
#include "../../Radiation_Model/source/config.h"
{% if initial_conditions.isothermal -%}
#define ISOTHERMAL
{%- endif %}
{% if initial_conditions.use_tabulated_gravity -%}
#define USE_TABULATED_GRAVITY
#define TABULATED_GRAVITY_FILE {{ general.tabulated_gravity_file }}
{%- endif %}

// **** Solver ****
#define EPSILON {{ solver.epsilon | is_required }}

// **** Grid ****
{% if grid.adapt -%}#define ADAPT{%- endif %}
#define MIN_CELLS {{ grid.minimum_cells | is_required }}
#define MAX_CELLS {{ grid.maximum_cells | is_required }}
#define MAX_REFINEMENT_LEVEL {{ grid.maximum_refinement_level | is_required }}
#define MIN_DS {{ grid.minimum_delta_s | is_required | units_filter('cm') }}
#define MAX_VARIATION {{ grid.maximum_variation | is_required + 1.0}}
#define MY_NEW_PARAM {{ grid.my_new_param }}
"""

In [7]:
loop_lengths = [40,
                #80,
                #150
               ]*u.Mm
for L in loop_lengths:
    tmp = config.copy()
    tmp['general']['loop_length'] = L
    c = Configure(tmp)
    c.setup_simulation(
        '/Users/willbarnes/Desktop/',
        base_path='/Users/willbarnes/Desktop/hydrad_test_L40/',
        name=f'hydrad_test_L{L.value:.0f}_foo',
        run_initial_conditions=False,
    )

In [8]:
! jupyter nbconvert --to script /Users/willbarnes/Documents/work/projects/research_notebook/HYDRAD/hydrad_tools_example.ipynb

[NbConvertApp] Converting notebook /Users/willbarnes/Documents/work/projects/research_notebook/HYDRAD/hydrad_tools_example.ipynb to script
[NbConvertApp] Writing 1126 bytes to /Users/willbarnes/Documents/work/projects/research_notebook/HYDRAD/hydrad_tools_example.py


In [8]:
config['general']['loop_length'] = 40*u.Mm
config['grid']['my_new_param']  = 100

In [3]:
c = Configure(config,)# templates={'initial_conditions.config.h': ic_template})

In [4]:
c.templates

['coefficients.cfg',
 'collisions.h',
 'heating.cfg',
 'heating.config.h',
 'hydrad.cfg',
 'hydrad.config.h',
 'initial_conditions.cfg',
 'initial_conditions.config.h',
 'radiation.config.h',
 'radiation.elements.cfg']

In [12]:
print(c.get_raw_template('collisions.h'))

// ****
// *
// * #defines for configuring the shortest collisional coupling timescale
// *
// * (c) Dr. Stephen J. Bradshaw
// *
// * Source code generated by hydrad_tools on {{ date }}
// *
// ****

// **** Physics ****
#define MINIMUM_COLLISIONAL_COUPLING_TIME_SCALE {{ general.minimum_collisional_coupling_timescale | units_filter('s') }}
{% if general.force_single_fluid -%}#define FORCE_SINGLE_FLUID{%- endif %}
// **** End of Physics ****



In [12]:
print(c.initial_conditions_header)

// ****
// *
// * #defines for configuring the hydrostatic model
// *
// * (c) Dr. Stephen J. Bradshaw
// *
// * Source code generated by hydrad_tools on 2018-09-01_14.18.30
// *
// ****

// **** Output ****
// **** End of Output ****

// **** Physics ****
#include "../../Radiation_Model/source/config.h"



// **** Solver ****
#define EPSILON 0.01

// **** Grid ****
#define ADAPT
#define MIN_CELLS 150
#define MAX_CELLS 30000
#define MAX_REFINEMENT_LEVEL 12
#define MIN_DS 1.0
#define MAX_VARIATION 1.1


In [15]:
c.env.list_templates()

['coefficients.cfg',
 'collisions.h',
 'heating.cfg',
 'heating.config.h',
 'hydrad.cfg',
 'hydrad.config.h',
 'initial_conditions.cfg',
 'initial_conditions.config.h',
 'radiation.config.h',
 'radiation.elements.cfg']

In [26]:
foo = c.env.get_template('initial_conditions.cfg')

In [30]:
with open(foo.filename, 'r') as f:
    print(f.read())

Initial_Conditions/profiles/initial.amr

{{ general.loop_length | is_required | units_filter('cm') }}
{{ general.loop_inclination | is_required | units_filter('deg') }}
{{ general.footpoint_height | is_required | units_filter('cm') }}

{{ initial_conditions.footpoint_temperature | is_required | units_filter('K') }}

{{ initial_conditions.footpoint_density | is_required | units_filter('cm^-3') }}

{{ initial_conditions.heating_location | is_required | units_filter('cm') }}
{{ initial_conditions.heating_scale_height | is_required | units_filter('cm') }}
{% if initial_conditions.isothermal -%}
0.0
0.0
1.0
1.0
{%- else -%}
{{ initial_conditions.heating_range_lower_bound | is_required | units_filter('erg cm^-3 s^-1') | log10_filter }}
{{ initial_conditions.heating_range_upper_bound | is_required | units_filter('erg cm^-3 s^-1') | log10_filter }}
{{ initial_conditions.heating_range_step_size | is_required }}
{{ initial_conditions.heating_range_fine_tuning | is_required }}
{%- endif %}

Confi

In [14]:
new_collisions = """// ****
// *
// * #defines for configuring the shortest collisional coupling timescale
// *
// * (c) Dr. Stephen J. Bradshaw
// *
// * Source code generated by hydrad_tools on {{ date }}
// *
// ****

// **** Physics ****
#define MINIMUM_COLLISIONAL_COUPLING_TIME_SCALE {{ general.minimum_collisional_coupling_timescale | units_filter('s') }}
{% if general.force_single_fluid -%}#define FORCE_SINGLE_FLUID{%- endif %}
// **** End of Physics ****
#define MY_NEW_PARAM {{ general.my_new_param }}"""

In [15]:
config['general']['my_new_param'] = 100

In [16]:
c_new = Configure(config, templates = {'collisions.h': new_collisions})

In [18]:
print(c_new.collisions_header)

// ****
// *
// * #defines for configuring the shortest collisional coupling timescale
// *
// * (c) Dr. Stephen J. Bradshaw
// *
// * Source code generated by hydrad_tools on 2018-09-01_15.14.17
// *
// ****

// **** Physics ****
#define MINIMUM_COLLISIONAL_COUPLING_TIME_SCALE 0.01

// **** End of Physics ****
#define MY_NEW_PARAM 100


In [7]:
class Foo:
    @property
    def foo(self,):
        return 'foo'
class Bar(Foo):
    @property
    def foo(self,):
        return 'bar'

In [8]:
Foo().foo

'foo'

In [9]:
Bar().foo

'bar'